In [70]:
from ngsolve import *
from netgen.geom2d import SplineGeometry
import netgen.gui

import matplotlib.pyplot as plt
from time import time

# Example 11.1

Simulate the Navier-Stokes equations on a channel with a step-like cross-section extension:
$\Omega = (0; 2]\times(0; 1) \cup (2; 10)\times(−1; 1)$. 

- On the left side we have a parabolic inflow profile,

- on the right side natural boundary conditions (outlet), thus $(-\nu\ \nabla u + pI)\cdot n = 0$.

- Everywhereelse we have wall boundary conditions (homogeneous Dirichlet bc). 

Choose a viscosity $\nu = 10^{-3}$ and a zero right hand side and use the Taylor-Hood element and the IMEX scheme as in the tutorial example navierstokes.py. 

Increase the inflow velocity and present an interpretation of your results. For this you might google "backward facing step, navier stokes". Further note, that you might need to adapt the time step size τ.

In [71]:
#Ω = (0, 2]×(0, 1) u (2, 10)×(−1, 1)
                  
geo = SplineGeometry()
# Left rectangle
a1 = geo.AppendPoint(0,0)
b1 = geo.AppendPoint(2,0)
c1 = geo.AppendPoint(2,1)
d1 = geo.AppendPoint(0,1)
# Right rectangle
a2 = geo.AppendPoint(2,-1)
b2 = geo.AppendPoint(10,-1)
c2 = geo.AppendPoint(10,1)
d2 = geo.AppendPoint(2,1)

p1_seq = [a1, b1, a2, b2, c2, c1, d1]
p2_seq = [b1, a2, b2, c2, c1, d1, a1]
bc_seq = ["wall"]*3 + ["outlet"] + ["wall"]*2 + ["inlet"]

for p1, p2, bc in zip(p1_seq, p2_seq, bc_seq):
    geo.Append(["line", p1, p2], leftdomain=1, rightdomain=0, bc=bc)

#geo.Append(["line", a1, b1], leftdomain=1, rightdomain=0, bc="wall")#
#geo.Append(["line", b1, a2], leftdomain=1, rightdomain=0, bc="wall")



#                                       bottom, right side, top, left side
#geo.AddRectangle((0, 0), (2, 1), bcs = ("wall", "outlet", "wall", "inlet"))
#geo.AddRectangle((2, -1), (10, 1), bcs = ("wall", "outlet", "wall", "inlet"))
#geo.AddCircle ( (0.2, 0.2), r=0.05, leftdomain=0, rightdomain=1, bc="cyl", maxh=0.02)
mesh = Mesh(geo.GenerateMesh(maxh=0.2))


Draw(mesh)

In [72]:
mesh.Curve(3)
Draw(mesh)

In [73]:
# Parameters
nu = Parameter(1e-3)
rhs = Parameter(0.0)


V = VectorH1(mesh,order=3, dirichlet="wall|inlet")
Q = H1(mesh,order=2)

X = FESpace([V,Q])

u,p = X.TrialFunction()
v,q = X.TestFunction()

stokes = nu * InnerProduct(grad(u), grad(v)) + (div(u)*q + div(v)*p) - 1e-10*p*q
a = BilinearForm(X)
a += stokes *dx
a.Assemble()

# nothing here ...
f = LinearForm(X)
#f += rhs*v*dx
f.Assemble()

In [86]:
# gridfunction for the solution
gfu = GridFunction(X)

# parabolic inflow at inlet:
width = 1
v_in = 10
uin = CoefficientFunction( (v_in*y*(1-y), 0) ) # uin...initial u
gfu.components[0].Set(uin, definedon=mesh.Boundaries("inlet"))


# solve Stokes problem for initial conditions:
inv_stokes = a.mat.Inverse(X.FreeDofs())

res = f.vec.CreateVector()
res.data = f.vec - a.mat*gfu.vec
gfu.vec.data += inv_stokes * res

# for visualization
print("Showing initial solution")
Draw (Norm(gfu.components[0]), mesh, "velocity", sd=3)

ERROR! Session/line number was not unique in database. History logging moved to new session 432
Showing initial solution


In [87]:
# timestepping parameters
tau = 0.001 #Parameter(0.0005)
tend = 10
#tau.Set()
# matrix for implicit Euler 
mstar = BilinearForm(X)
mstar += SymbolicBFI(u*v + tau*stokes)
mstar.Assemble()
inv = mstar.mat.Inverse(X.FreeDofs(), inverse="sparsecholesky")

# the non-linear term 
conv = BilinearForm(X, nonassemble = True)
conv += (grad(u) * u) * v * dx

# implicit Euler/explicit Euler splitting method:
t = 0
rt = time()
with TaskManager():
    while t < tend:
        print ("t=", t, end="\r")

        conv.Apply(gfu.vec, res)
        res.data += a.mat*gfu.vec
        gfu.vec.data -= tau * inv * res    

        t = t + tau
        Redraw()
rt = time() - rt
print()
rt

t= 9.999999999999897676 0.8840000000000007 1.033999999999997 1.1729999999999816 1.6559999999999284 1.6919999999999245 1.7579999999999172 1.8849999999999032 2.1669999999998724 2.2019999999998685 2.205999999999868 2.270999999999861 2.5909999999998257 2.650999999999819 2.7379999999998095 3.0669999999997732 3.0999999999997696 3.1939999999997593 3.7019999999997033 4.048999999999687 4.216999999999743 4.243999999999752 4.304999999999772 4.344999999999786 4.863999999999959 4.883999999999966 4.887999999999967 4.908999999999974 5.314000000000109 5.328000000000114 5.339000000000118 5.392000000000135 5.682000000000232 5.782000000000266 5.790000000000268 5.817000000000277 5.82600000000028 5.875000000000297 6.046000000000354 6.053000000000356 6.1950000000004035 6.2010000000004055 6.381000000000466 6.387000000000468 7.32200000000078 7.328000000000782 8.054000000000975 8.062000000000971 9.270000000000302 9.277000000000298


215.47478699684143

1591618581.2876022

Do mochts an Wüabö!